##### Copyright 2025 Google LLC.

In [7]:
#!pip install PyPDF2
from PyPDF2 import PdfReader
import re

# Load the full PDF
pdf_path = "/content/FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf"
reader = PdfReader(pdf_path)

# Combine all text for processing
all_text = ""
for page in reader.pages:
    all_text += page.extract_text() + "\n"

# Define regex pattern to extract tabular data
line_item_pattern = re.compile(
    r"(?P<description>.+?)\s+(?P<quantity>\d+\.\d{2})\s+(?P<unit>[A-Z]+)\s+"
    r"(?P<tax>\d+\.\d{2})\s+(?P<op>\d+\.\d{2})\s+(?P<rcv>\d+\.\d{2})\s+"
    r"(?P<age_life>\d+\/(?:\d+|NA)[^\s]*)\s+(?P<cond>Avg\.)\s+"
    r"(?P<dep_pct>\d+\.?\d*%)\s+\((?P<deprec>\d+\.\d{2})\)\s+(?P<acv>\d+\.\d{2})"
)

# Extract all matches
matches = list(line_item_pattern.finditer(all_text))

# Build JSON-like structure
records = []
for idx, match in enumerate(matches, 1):
    groups = match.groupdict()
    record = {
        "Serial": idx,
        "Description": groups["description"].strip(),
        "Quantity": float(groups["quantity"]),
        "Unit": groups["unit"],
        "Tax": float(groups["tax"]),
        "O&P": float(groups["op"]),
        "RCV": float(groups["rcv"]),
        "Age/Life": groups["age_life"],
        "Cond.": groups["cond"],
        "Dep%": groups["dep_pct"],
        "Deprec.": float(groups["deprec"]),
        "ACV": float(groups["acv"])
    }
    records.append(record)

#records[:5]  # Show sample of first 5 items


In [11]:
import os
import json
import re
from datetime import datetime
from trp.t_pipeline import pipeline_merge_tables
import trp.trp2 as t2
from textractcaller.t_call import call_textract, Textract_Features
from textractprettyprinter.t_pretty_print import Textract_Pretty_Print, get_string, get_tables_string, Pretty_Print_Table_Format
from trp.trp2 import TDocument, TDocumentSchema
from trp.t_tables import MergeOptions, HeaderFooterType
import boto3
import fitz
import pandas as pd
from trp import Document
from textractprettyprinter.t_pretty_print import convert_table_to_list
from IPython.display import display


textract_client = boto3.client('textract', region_name='region',aws_access_key_id='', aws_secret_access_key='')



import boto3
s3bucket = boto3.resource(service_name='s3', region_name='ap-south-1', aws_access_key_id='', aws_secret_access_key='')

#Filename = "D:\\downloads\Working PDF Files\\Working PDF Files\\1. CIC insurance\\extracted_FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf"
Filename="/content/FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf"
key = "FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf"
s3bucket.Bucket('awstextractpdfuploads').upload_file(Filename=Filename, Key=key)

s3_uri_of_documents = "s3://awstextractpdfuploads/FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf"
textract_json_cic_full_insurance = call_textract(input_document=s3_uri_of_documents, features=[Textract_Features.TABLES], boto3_textract_client = textract_client)

# use this textract_json_cic_full_insurance object in code



import traceback
import json


def allowed_file(filename):
    ALLOWED_EXTENSIONS = {'pdf'}
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def remove_leading_numbers(text):
    return re.sub(r'^\d+\.\s*', '', text)

def extract_pages_with_keywords(directory_path, file_name, unique_no):
    try:
        # Define keyword groups

        GROUP_1 = {"Quantity", "QTY", "Measurement"}
        GROUP_2 = {"Price", "Unit", "UNIT PRICE", "Replace", "Remove"}
        GROUP_3 = {"Total", "Total Price", "RCV", "RC"}

        pdf_path = directory_path + file_name
        doc = fitz.open(pdf_path)
        selected_pages = []

        # Check each page for the required keywords
        for page_num in range(len(doc)):
            text = doc[page_num].get_text("text").lower()
            if (any(kw.lower() in text for kw in GROUP_1) and
                    any(kw.lower() in text for kw in GROUP_2) and
                    any(kw.lower() in text for kw in GROUP_3)):
                selected_pages.append(page_num + 1)  # Page numbers are 1-based
                #print('#' * 100 + str(' page ') + str(page_num))
        if not selected_pages:
            print("No matching pages found.")
            return None

        output_pdf_path = directory_path + str(unique_no) + '_' + file_name
        new_doc = fitz.open()
        #print(output_pdf_path)
        for page_num in selected_pages:
            new_doc.insert_pdf(doc, from_page=page_num - 1, to_page=page_num - 1)

        new_doc.save(output_pdf_path)
        new_doc.close()

        print(f"Extracted PDF saved as: {output_pdf_path}")
    except Exception as e:
        print(e)
    return output_pdf_path

def merge_and_clean_data(data):
    output = []
    temp_desc = ""
    temp_quantity = ""
    last_valid_acv_item = None
    try:
        KEY_VARIANTS = {
            'quantity': ['quantity', 'qty'],
            'unit': ['unit', 'unit price'],
            'tax': ['tax', 'tax percent', 'tax value']
        }

        def find_flexible_key(item, standard_key):
            """Find the key in `item` that matches any known variant for the given standard key."""
            variants = KEY_VARIANTS.get(standard_key, [])
            for var in variants:
                for key in item:
                    if var.lower() in key.lower():
                        return key
            return None

        #print('EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE')
        #print(data)
        #print('DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDd')
        for item in data:
            #print('########## '+ str(item))
            if not item["ACV"].strip():  # If ACV is empty
                # Merge all non-empty values into DESCRIPTION
                combined_text = " ".join(v.strip() for k, v in item.items() if v.strip())
                temp_desc += " " + combined_text if combined_text else ""
                temp_desc = temp_desc.strip()
            else:  # ACV exists
                if temp_desc:  # Merge previous descriptions into this item
                    item["DESCRIPTION"] = temp_desc
                    item["DESCRIPTION"] = item["DESCRIPTION"].strip()
                    temp_desc = ""

            if item["ACV"].strip() or ('Total:' in item.get('QUANTITY', '') or 'Totals:' in item.get('QUANTITY', '')):
                output.append(item)
            last_valid_acv_item = item  # Update last valid ACV item

            # Find flexible keys
            '''qty_key = find_flexible_key(item, 'quantity')
            unit_key = find_flexible_key(item, 'unit')
            tax_key = find_flexible_key(item, 'tax')

            qty_list = item.get(qty_key, '').split(' ') if qty_key else []
            unit_list = item.get(unit_key, '').split(' ') if unit_key else []
            tax_list = item.get(tax_key, '').split(' ') if tax_key else []


            # Add leftover parts to DESCRIPTION
            if len(qty_list) > 2:
                item['DESCRIPTION'] += ' ' + ' '.join(qty_list[:-2])
            if len(unit_list) > 1:
                item['DESCRIPTION'] += ' ' + ' '.join(unit_list[:-1])
            if len(tax_list) > 1:
                item['DESCRIPTION'] += ' ' + ' '.join(tax_list[:-1])

            item['DESCRIPTION'] = item['DESCRIPTION'].strip()'''

            #print('$$$$$$$$$$$$$$$$$$$$$ '+ str(item))
    except Exception as e:
        print(e)

    return output


def merge_entries(data):
    merged_list = []
    skip_next = False

    for i in range(len(data) - 1):
        if skip_next:
            skip_next = False
            continue
        try:
            current = data[i]
            next_entry = data[i + 1]
            last_key = next(reversed(current))
            # If the current entry has an empty ACV, merge its values into next entry's description
            if not current[last_key]:
                merged_description = " ".join(filter(None, [current[key] for key in current]))  # Concatenate all values
                merged_entry = next_entry.copy()
                merged_entry['DESCRIPTION'] = merged_description

                merged_list.append(merged_entry)
                skip_next = True  # Skip the next entry since it's merged
            else:
                merged_list.append(current)
        except:
            pass
    # Add the last entry if it wasn't merged
    if not skip_next:
        merged_list.append(data[-1])

    return merged_list


def clean_description(data):
    try:
        # Process each room in the data
        quantity_keys = {"QTY", "QUANTITY", "MEASUREMENT"}

        def find_QTY_key(d):
            """Find the key in the dictionary that matches QTY, QUANTITY, or MEASUREMENT."""
            for key in d:
                if key.upper() in quantity_keys:
                    return key
            return None  # No matching key found

        for room, items in data.items():
            for item in items:
                QTY_key = find_QTY_key(item.keys())
                # Clean numbered descriptions
                if "DESCRIPTION" in item:
                    # Remove number prefix (e.g., "20. ", "21. ")
                    if item["DESCRIPTION"].strip():
                        parts = item["DESCRIPTION"].split(".")
                        if len(parts) > 1 and parts[0].strip().isdigit():
                            item["DESCRIPTION"] = ".".join(parts[1:]).strip()

                # Handle empty descriptions with data in QUANTITY
                if "DESCRIPTION" in item and not item["DESCRIPTION"].strip() and QTY_key in item:
                    quantity_parts = item[QTY_key].split()
                    if len(quantity_parts) >= 2:
                        # Extract description and quantity
                        description_parts = []
                        quantity_value = None
                        unit = None

                        for part in quantity_parts:
                            if any(char.isdigit() for char in part):
                                quantity_value = part
                                if unit:
                                    break
                            else:
                                if quantity_value:
                                    unit = part
                                else:
                                    description_parts.append(part)

                        if description_parts:
                            item["DESCRIPTION"] = " ".join(description_parts)
                            if quantity_value and unit:
                                item[QTY_key] = f"{quantity_value} {unit}"

        return data
    except Exception as e:
        print(e)
        print(traceback.format_exc())

# Function to get unique keys from result 2 that aren't in result 1
def compare_and_get_unique_keys(result1, result2):
    result3 = {}
    try:
        key_map = {
            "quantity": "measurement", "qty": "measurement", "measurement": "measurement",
            "price": "unit", "unit": "unit", "replace": "unit", "remove": "unit", "unit price": "unit"
        }

        def normalize_key(key):
            return key_map.get(key.lower(), key)  # Convert to lowercase and map if applicable

        # Get all sections from result2
        for section in result2.keys():
            # Skip if section exists in result1
            if section not in result1:
                continue

            existing_keys = [normalize_key(k) for k in result1[section][0].keys()]
            # Get all unique keys from the items in this section
            unique_keys = list()

            for item in result2[section]:
                for key in item.keys():
                    norm_key = normalize_key(key)
                    if norm_key not in existing_keys:
                        if key not in unique_keys:
                            unique_keys.append(key)

            if unique_keys:
                result3[section] = unique_keys
    except Exception as e:
        print(e)

    return result3


# Function to compare two JSONs dynamically based on the first key
def compare_json(result_1, result_2):
    output = {}
    quantity_keys = {"QTY", "QUANTITY", "MEASUREMENT"}
    keys_to_check = {"PRICE", "UNIT", "REPLACE"}  # Standardized keys

    def find_quantity_key(d):
        """Find the key in the dictionary that matches QTY, QUANTITY, or MEASUREMENT."""
        for key in d:
            if key.upper() in quantity_keys:
                return key
        return None  # No matching key found

    def extract_float(value):
        """Extracts a float from a string if it contains a number, otherwise returns None."""
        if isinstance(value, (int, float)):
            return float(value)  # Already a number
        elif isinstance(value, str):
            match = re.search(r"[-+]?\d*\.\d+|\d+", value)  # Find numeric value
            return float(match.group()) if match else None
        return None  # If value is not a valid type

    def find_unit_key(d):
        """Find a key that matches the defined set."""
        for key in d:
            if key.upper() in keys_to_check:
                return key
        return None  # No matching key found
    try:
        for key_2, records_2 in result_2.items():
            table_found = "Yes" if key_2 in result_1 else "No"
            result_list = []

            for record_2 in records_2:
                try:
                    record_2.update({"STATUS": {}})
                    record_match = "No"

                    first_key_2 = next(iter(record_2), None)
                    first_value_2 = record_2[first_key_2] if first_key_2 else None

                    # Check if the key exists in result_1
                    if table_found == "Yes":
                        # print(result_1[key_2])
                        for record_1 in result_1[key_2]:
                            # print(record_1.keys())
                            first_key_1 = next(iter(record_1), None)  # Get first key dynamically
                            first_value_1 = record_1[first_key_1] if first_key_1 else None

                            # Compare first values
                            if first_value_1 == first_value_2:
                                key1 = find_quantity_key(record_1)
                                key2 = find_quantity_key(record_2)

                                fk1 = find_unit_key(record_1)
                                fk2 = find_unit_key(record_2)

                                record_match = "Yes"
                                if key1 and key2:
                                    value1, value2 = record_1[key1], record_2[key2]

                                    if value1 == value2:
                                        #print(f"Values match: {value1}")
                                        record_2['STATUS'][key2] = 'Yes'
                                    else:
                                        record_2['STATUS'][key2] = 'No'
                                        #print(f"Values do not match: {value1} (dict1) vs {value2} (dict2)")
                                #else:
                                #    print("No common quantity key found in both dictionaries.")

                                if fk1 and fk2:
                                    if fk1 == 'REPLACE':
                                        value1 = extract_float(record_1[fk1]) + extract_float(record_1['REMOVE'])
                                        value2 = extract_float(record_2[fk2])
                                    elif fk2 == 'REPLACE':
                                        value1 = extract_float(record_1[fk1])
                                        value2 = extract_float(record_2[fk2]) + extract_float(record_2['REMOVE'])
                                    else:
                                        value1, value2 = extract_float(record_1[fk1]), extract_float(record_2[fk2])

                                    if value1 is None or value2 is None:
                                        #print(
                                        #    f"Could not extract valid numbers: {dict1[fk1]} (dict1) vs {dict2[fk2]} (dict2)")
                                        if fk2 == 'REPLACE':
                                            record_2['STATUS'][fk2] = 'No'
                                            record_2['STATUS']['REMOVE'] = 'No'
                                        else:
                                            record_2['STATUS'][fk2] = 'No'
                                    elif value1 == value2:
                                        #print(f"Values match: {value1}")
                                        record_2['STATUS'][fk2] = 'Yes'
                                    #else:
                                    #    print(f"Values do not match: {value1} (dict1) vs {value2} (dict2)")

                                break  # Stop once a match is found

                    # Add match results
                    record_2['STATUS']["TABLE_MATCH"] = table_found
                    record_2['STATUS']["RECORD_MATCH"] = record_match
                    result_list.append(record_2)
                except Exception as e:
                    print(e)

            output[key_2] = result_list
    except Exception as e:
        print(e)
    return output



def remove_images_pdf(directoryPath, fileName):
    try:
        print(directoryPath)
        pdf_path = directoryPath + fileName
        print(pdf_path)
        doc = fitz.open(pdf_path)
        large_image_pages = []

        # Minimum dimensions for what we consider a "large" image (in pixels)
        MIN_WIDTH = 200
        MIN_HEIGHT = 200

        for page_num in range(len(doc)):
            page = doc[page_num]
            images = page.get_images(full=True)

            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = doc.extract_image(xref)

                if base_image:
                    # Get image dimensions
                    width = base_image["width"]
                    height = base_image["height"]

                    # Check if image meets minimum size requirements
                    if width >= MIN_WIDTH and height >= MIN_HEIGHT:
                        large_image_pages.append(page_num + 1)  # Pages are 1-indexed
                        break  # One large image is enough to mark the page

        print("Pages with large images:", large_image_pages)
        output_pdf = directoryPath + "extracted_" + fileName

        # Create new PDF without pages containing large images
        doc = fitz.open(pdf_path)
        doc.select([i for i in range(len(doc)) if (i + 1) not in large_image_pages])
        doc.save(output_pdf)
        doc.close()
        print(f"New PDF saved as {output_pdf}")

    except Exception as e:
        print(e)
    return "success"

def process_items_old(final_list):
    processed_items = []
    try:
        quantity_pattern = re.compile(r'(?P<quantity>\d+(?:\.\d+)?)\s*(?P<unit>LF|SF|SQ|EA|HR)')
        quantity_keys = {"QTY", "QUANTITY", "MEASUREMENT"}
        def find_quantity_key(data):
            for key in quantity_keys:
                if key in data:
                    return key  # Return the found key
            return None

        for items in final_list:
            for item in items:
                description = item.get("DESCRIPTION", "").strip()
                qty_key = find_quantity_key(item)
                quantity = item.get(qty_key, "").strip()

                match = quantity_pattern.search(quantity)
                if match:
                    extracted_quantity = match.group("quantity") + " " + match.group("unit")
                    remaining_text = quantity.replace(match.group(0), "").strip()

                    if not description:
                        description = remaining_text
                        quantity = extracted_quantity
                    else:
                        quantity = extracted_quantity
                        description = description + " " + remaining_text if remaining_text else description

                item["DESCRIPTION"] = description
                item[qty_key] = quantity
    except Exception as e:
        print(e)
    return final_list

def process_items(final_list):
    processed_items = []
    try:
        quantity_pattern = re.compile(r'(?P<quantity>\d+(?:\.\d+)?)\s*(?P<unit>LF|SF|SQ|EA|HR)')

        quantity_keys = {"QTY", "QUANTITY", "MEASUREMENT"}
        unit_keys = {"UNIT", "UNIT PRICE"}
        tax_keys = {"TAX"}

        def find_quantity_key(data):
            for key in quantity_keys:
                if key in data:
                    return key  # Return the found key
            return None

        def find_unit_key(data):
            for key in unit_keys:
                if key in data:
                    return key  # Return the found key
            return None

        for items in final_list:
            for item in items:
                description = item.get("DESCRIPTION", "").strip()
                qty_key = find_quantity_key(item)
                unit_key = find_unit_key(item)
                unitvalue = ''
                unit_description = ''
                quantity = item.get(qty_key, "").strip()
                unit_value = item.get(unit_key, "").strip()
                tax_value = item.get('TAX', "").strip()

                match = quantity_pattern.search(quantity)
                if match:
                    extracted_quantity = match.group("quantity") + " " + match.group("unit")
                    remaining_text = quantity.replace(match.group(0), "").strip()

                    if not description:
                        description = remaining_text
                        quantity = extracted_quantity
                    else:
                        quantity = extracted_quantity
                        description = description + " " + remaining_text if remaining_text else description

                if unit_value:
                    unit_pattern = re.match(r'^(.*?)(\d+(?:\.\d+)?)?\s*$', unit_value.strip())
                    if unit_pattern:
                        unit_description = unit_pattern.group(1).strip()
                        value_str = unit_pattern.group(2)
                        unitvalue = float(value_str) if value_str else None

                if tax_value is not None:
                    tax_pattern = re.match(r'^(.*?)(\d+(?:\.\d+)?)?\s*$', tax_value.strip())
                    if tax_pattern:
                        tax_description = tax_pattern.group(1).strip()
                        value_str2 = tax_pattern.group(2)
                        tax_value = float(value_str2) if value_str2 else None

                if unit_description:
                    description = description + ' ' + unit_description

                if tax_description:
                    description = description + ' ' + tax_description

                item["DESCRIPTION"] = description
                item[qty_key] = quantity

                if unitvalue:
                    item[unit_key] = str(unitvalue)
                #if tax_value:
                item['TAX'] = str(tax_value)

    except Exception as e:
        print(e)
    return final_list

def add_record_ids(data):
    result = {}
    count = 1
    try:
        for room, items in data.items():
            result[room] = []
            for idx, item in enumerate(items, 1):
                new_item = item.copy()
                new_item["record_id"] = f"{count}"
                count += 1
                result[room].append(new_item)
    except Exception as e:
        print(e)
    return result

def transform_construction_data22(data):
    result = {}
    current_section = []

    for section in data:
        section_name = None

        try:
            last_record = section[-1]
            # Check if last_record has relevant keys
            if any(key in last_record for key in ["DESCRIPTION", "QUANTITY", "QTY"]):
                # Check if the last record is a total record
                if any("Total:" in (last_record.get(key) or "") or "Totals:" in (last_record.get(key) or "")
                       for key in ["DESCRIPTION", "QUANTITY"]):

                    section_name = "Unknown Section"

                    try:
                        # If no valid name in DESCRIPTION, check QUANTITY
                        if section_name == "Unknown Section" and "QUANTITY" in last_record and isinstance(last_record["QUANTITY"], str):
                            qty_clean = last_record["QUANTITY"].replace("Totals:", "").replace("Total:", "").strip()
                            #print('qty clean '+ qty_clean)
                            if qty_clean:
                                section_name = qty_clean

                        # Extract section name from DESCRIPTION if valid
                        if section_name == "Unknown Section" and "DESCRIPTION" in last_record and isinstance(last_record["DESCRIPTION"], str):
                            desc_clean = last_record["DESCRIPTION"].replace("Totals:", "").replace("Total:", "").strip()
                            #print('desc  clean '+ desc_clean)
                            if desc_clean and len(desc_clean.split()) <= 5:  # Ensure it's not a full sentence
                                section_name = desc_clean

                    except Exception as e:
                        print(f"Error extracting section name: {e}")

                    # Print debug information
                    print(f"Section Name Identified: {section_name}")

                    # Store pending records before starting new section
                    if current_section:
                        if section_name not in result:
                            result[section_name] = []
                        result[section_name].extend(current_section)
                        current_section = []

                    # Store current section's records except last (total record)
                    if section_name not in result:
                        result[section_name] = []
                    result[section_name].extend(section[:-1])

                else:
                    # If no total record, accumulate records in current_section
                    current_section.extend(section)

        except Exception as e:
            print(traceback.format_exc())
            print(f"Exception occurred: {e}")

    # Assign any remaining records to the last section
    if current_section and result:
        last_section = list(result.keys())[-1]
        result[last_section].extend(current_section)

    return result


def transform_construction_data2(data):
    result = {}
    current_section = []

    KEY_VARIANTS = {
        'quantity': ['quantity', 'qty'],
        'unit': ['unit', 'unit price'],
        'tax': ['tax', 'tax percent', 'tax value']
    }

    def find_flexible_key(item, standard_key):
        """Find the key in `item` that matches any known variant for the given standard key."""
        variants = KEY_VARIANTS.get(standard_key, [])
        for var in variants:
            for key in item:
                if var.lower() in key.lower():
                    return key
        return None

    def is_non_numeric_string(value):
        try:
            float(value)
            return False  # It's numeric (even if in string form)
        except (ValueError, TypeError):
            return isinstance(value, str)

    for section in data:
        section_name = None

        try:
            last_record = section[-1]
            # Check if last_record has relevant keys
            if any(key in last_record for key in ["DESCRIPTION", "QUANTITY", "QTY"]):
                #print(last_record)
                qty_key = find_flexible_key(last_record, 'quantity')
                unit_key = find_flexible_key(last_record, 'unit')

                # Check if the last record is a total record
                if any("Total:" in (last_record.get(key) or "") or "Totals:" in (last_record.get(key) or "")
                       for key in ["DESCRIPTION", "QUANTITY", "QTY"]):

                    section_name = "Unknown Section"

                    try:
                        # If no valid name in DESCRIPTION, check QUANTITY
                        if section_name == "Unknown Section" and qty_key in last_record and isinstance(last_record[qty_key], str):
                            #print('in one')
                            desc_clean = last_record["DESCRIPTION"].replace("Totals:", "").replace("Total:", "").strip()
                            qty_clean = last_record[qty_key].replace("Totals:", "").replace("Total:", "").strip()
                            unit_value = last_record.get(unit_key, '').strip()
                            unit_type = is_non_numeric_string(unit_value)
                            #print('desc clean '+ desc_clean)
                            #print('qty clean' + qty_clean)
                            #print('unit value ' + unit_value)

                            if desc_clean:
                                section_name = desc_clean
                            if qty_clean:
                                if section_name != 'Unknown Section':
                                    section_name = section_name + ' ' + qty_clean
                                else:
                                    section_name = qty_clean
                            if unit_type and unit_value:
                                if section_name != 'Unknown Section':
                                    section_name = section_name + ' ' + unit_value
                                else:
                                    section_name = unit_value


                        # Extract section name from DESCRIPTION if valid
                        if section_name == "Unknown Section" and "DESCRIPTION" in last_record and isinstance(last_record["DESCRIPTION"], str):
                            desc_clean = last_record["DESCRIPTION"].replace("Totals:", "").replace("Total:", "").strip()
                            qty_value = last_record.get(qty_key, '').strip()
                            qty_type = is_non_numeric_string(qty_value)
                            unit_value = last_record.get(unit_key, '').strip()
                            unit_type = is_non_numeric_string(unit_value)

                            if desc_clean and len(desc_clean.split()) <= 5:  # Ensure it's not a full sentence
                                section_name = desc_clean
                            if qty_type and qty_value:
                                if section_name != 'Unknown Section':
                                    section_name = section_name + ' ' + qty_value
                                else:
                                    section_name = qty_value
                            if unit_type and unit_value:
                                if section_name != 'Unknown Section':
                                    section_name = section_name + ' ' + unit_value
                                else:
                                    section_name = unit_value
                    except Exception as e:
                        print(f"Error extracting section name: {e}")

                    # Print debug information
                    print(f"Section Name Identified: {section_name}")
                    # Store pending records before starting new section
                    if current_section:
                        if section_name not in result:
                            result[section_name] = []
                        result[section_name].extend(current_section)
                        current_section = []

                    # Store current section's records except last (total record)
                    if section_name not in result:
                        result[section_name] = []
                    result[section_name].extend(section[:-1])

                else:
                    # If no total record, accumulate records in current_section
                    current_section.extend(section)

        except Exception as e:
            print(traceback.format_exc())
            print(f"Exception occurred: {e}")

    # Assign any remaining records to the last section
    if current_section and result:
        last_section = list(result.keys())[-1]
        result[last_section].extend(current_section)

    return result

import traceback
def PrettyPrintTablesTextract(textract_json):
    df = None
    #table_count = 0
    tdoc = Document(textract_json)
    final_list = []
    quantity_keys = {"QTY", "QUANTITY", "MEASUREMENT"}

    def find_QTY_key(d):
        """Find the key in the dictionary that matches QTY, QUANTITY, or MEASUREMENT."""
        for key in d:
            if key.upper() in quantity_keys:
                return key
        return None  # No matching key found
    def starts_with_integer(text):
        return bool(re.match(r'^\d+', text.strip())) if text else False

    def remove_unwanted_description(converted_data):
        try:
            converted_data_output = []
            for item in converted_data:
                if starts_with_integer(item.get('DESCRIPTION', '').strip()):
                    converted_data_output.append(item)
                else:

                    if item.get('DESCRIPTION', ''):
                        if ('Total:' or 'Totals:') in item.get('DESCRIPTION', ''):
                            converted_data_output.append(item)
                        #else:
                        #    print('in else')
                        #    print('removed '+ str(item))
                    else:
                        #print('in description empty')
                        if starts_with_integer(item.get('QUANTITY', '').strip()):
                            converted_data_output.append(item)
                        else:
                            if 'Total:' in item.get('QUANTITY', '') or 'Totals:' in item.get('QUANTITY', ''):
                                converted_data_output.append(item)
                            #else:
                            #    print('in else21')

                            #    print('removed21 '+ str(item))

            return converted_data_output
        except Exception as e:
            print(e)
            print(traceback.format_exc())
            return converted_data

    for page in tdoc.pages:
        for table in page.tables:
            desc_exist = True
            #table_count += 1
            df = pd.DataFrame(convert_table_to_list(trp_table=table))
            #display(df)
            json_data = df.to_json(orient='records')
            json_data1 = json.loads(json_data)
            headers = list(json_data1[0].values())
            #print(headers)
            headers = [header.strip().upper() for header in headers]
            #print(headers)
            QTY_key = find_QTY_key(headers)
            #print('QTY_key ' + str(QTY_key))

            if QTY_key not in headers:
                #print('in not in headers')
                # quantity_row_idx = df[df.isin(["QUANTITY ", "QTY ", "MEASUREMENT "]).any(axis=1)].index.min()
                # quantity_row_idx = df[df.astype(str).apply(lambda x: x.str.upper()).isin(["QUANTITY ", "QTY ", "MEASUREMENT "]).any(axis=1)].index.min()
                quantity_row_idx = df[df.astype(str).apply(lambda x: x.str.upper().str.strip()).isin(
                    ["QUANTITY", "QTY", "MEASUREMENT"]).any(axis=1)].index.min()

                if pd.notna(quantity_row_idx):
                    df = df.drop(index=0).reset_index(drop=True)
                    json_data = df.to_json(orient='records')
                    json_data1 = json.loads(json_data)
                    headers = list(json_data1[0].values())

            headers = [header.strip().upper() for header in headers]
            #print(headers)

            totals_row_idx = None

            patterns = [r'(?i)^total:', r'(?i)^totals:', r'(?i)total:\s', r'(?i)totals:\s']
            # Check each row for the patterns
            for idx, row in df.iterrows():
                for col in row.index:
                    cell_value = str(row[col]).strip()
                    # Check if cell contains "Total:" or "Totals:"
                    if any(pd.Series(cell_value).str.contains(pattern, regex=True).any() for pattern in
                           patterns):
                        if totals_row_idx is None or idx < totals_row_idx:
                            totals_row_idx = idx

            # If a "Total:" or "Totals:" row was found, remove all rows after it
            if totals_row_idx is not None:
                df = df.loc[:totals_row_idx].copy()
            json_data = df.to_json(orient='records')
            json_data1 = json.loads(json_data)
            #print('#' * 100 + str(' after remove totals below'))
            #print(df)

            table_data = json_data1[1:]
            if headers[0] == '':
                headers[0] = 'DESCRIPTION'
                desc_exist = False
            elif headers[0] != 'DESCRIPTION':
                desc_exist = False
                table_data = []
                headers.insert(0, 'DESCRIPTION')
                for item in json_data1[1:]:
                    new_item = {'0': ''}  # Add new empty key '0'

                    # Shift all existing keys up by 1
                    for key, value in item.items():
                        new_key = str(int(key) + 1)  # Convert to int, add 1, then back to str
                        new_item[new_key] = value

                    table_data.append(new_item)

            #print('after added ' + str(headers))

            converted_data = []
            for row in table_data:
                # Ignore rows that don't have valid numbers in relevant fields
                if 'REPLACE' in headers:
                    if not any(row.get(str(i), "").strip().replace(",", "").replace(".", "").isdigit() for i in
                               range(1, 8)):
                        continue
                if ('DESCRIPTION' in headers) and any(h in headers for h in ['QTY', 'MEASUREMENT', 'QUANTITY']) \
                        and any(h in headers for h in ['UNIT', 'PRICE', "UNIT PRICE", 'REPLACE']):
                    #print('in if condition success')
                    converted_data.append(
                        {headers[i].strip(): row.get(str(i), "").strip() for i in range(len(headers))})

            converted_data = [item for item in converted_data if item != {'': ''}]
            # converted_data = [{k: v for k, v in item.items() if v.strip() != ""} for item in converted_data]
            if converted_data:
                print(df)

            total_entries = len(converted_data)
            qty_starting_with_int = 0
            #print('cd ' + str(converted_data))
            count_starting_with_int = sum(
                1 for item in converted_data
                if starts_with_integer(item.get('DESCRIPTION', ''))
            )
            if count_starting_with_int == 0:
                desc_exist = sum(1 for item in converted_data if item.get('DESCRIPTION', ''))
                if not desc_exist:
                    qty_starting_with_int = sum(
                        1 for item in converted_data
                        if starts_with_integer(item.get('QUANTITY', ''))
                    )
            if qty_starting_with_int:
                converted_data = remove_unwanted_description(converted_data)
            #print('cd3 ' + str(converted_data))
            #print('integer '+ str(count_starting_with_int))
            try:
                if converted_data and count_starting_with_int:
                    if 'ACV' in converted_data[0].keys():
                        #print('in if')
                        percentage_with_description1 = (count_starting_with_int / total_entries) * 100
                        #print('in '+ str(percentage_with_description1))
                        if percentage_with_description1 >= 20:
                            converted_data = remove_unwanted_description(converted_data)
                    else:
                        print('in elseeee')
            except:
                print('in except')
                print(traceback.format_exc())
                pass

            #print('cd2 ' + str(converted_data))
            final_list2 = []
            if converted_data:
                #print(desc_exist)
                #print(converted_data)
                if desc_exist == False:
                    #print('###########2 converted data' + str(converted_data))

                    # Count entries with non-empty descriptions
                    non_empty_descriptions = sum(1 for item in converted_data if item.get('ACV'))
                    # Calculate percentage
                    percentage_with_description = (non_empty_descriptions / total_entries) * 100
                    #print('pwd '+str(percentage_with_description))
                    if percentage_with_description <= 90:
                        #final_list2 = merge_entries(converted_data)
                        final_list2 = merge_and_clean_data(converted_data)
                    else:
                        #print('cd2 ' + str(converted_data))
                        final_list2 = converted_data
                else:
                    #print('in else')
                    # Example usage:
                    final_list2 = merge_entries(converted_data)
                #print(final_list2)

                final_list.append(final_list2)
            #print('#' * 50)
        # print(final_list)
    return final_list


final_list = PrettyPrintTablesTextract(textract_json_cic_full_insurance)
#print('#' * 100)
#print(json.dumps(final_list, indent=4))
#print('final list')

#print(final_list)
#print('#' * 100)

final_list = process_items(final_list)  # qa

results1 = transform_construction_data2(final_list)  # qa
results1 = {key: value for key, value in results1.items() if value}  # qa


results1 = clean_description(results1)  # qa
print(json.dumps(results1, indent=4))
#print(results1.indent(4))
#results3 = compare_and_get_unique_keys(results1, results2)
#print(results3)

#results2 = compare_json(results1, results2)

#results2 = add_record_ids(results2)


#print(results1)
#print(results2)

S3UploadFailedError: Failed to upload /content/FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf to awstextractpdfuploads/FINALDRAFTWITHAGELIFEANDCONDITIONReport32825_66.pdf: An error occurred (AuthorizationHeaderMalformed) when calling the PutObject operation: The authorization header is malformed; a non-empty Access Key (AKID) must be provided in the credential.